# Network Analysis of Functional Connectivity

In this tutorial, we'll apply simply graph theoretical techniques to characterize our functional connectivity data we generated in tutorial 10. We'll discuss how our connectivity matrix maps onto a graph, and what types of measures we can pull out of it in order to understand the underlying connectivity structure. 

Without a doubt, the foundational paper for the analyses we'll perfom here is [Rubinov and Sporns (2010)](https://www.sciencedirect.com/science/article/abs/pii/S105381190901074X?via%3Dihub). They describe a variety of ways in which we can analyze brain networks beyond applying simple statistics to connectivity matrices (e.g., mass univariate testing). Definitely required reading if you are interested in functional connectivity. Another excellent resource is [Fundamentals of Brain Network Analysis](https://www.elsevier.com/books/fundamentals-of-brain-network-analysis/fornito/978-0-12-407908-3) by Bullmore et al.

Here we'll also make use of [Networkx](https://networkx.github.io/documentation/stable/index.html), which let's us visualize graphs. In combination with Brain Connectivity Toolbox (BCT), we have a core toolbox for network neuroscience at our disposal.

In [2]:
import numpy as np
import pandas as pd
from scipy import spatial, stats, cluster
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import plotting, input_data
from nilearn.datasets import fetch_atlas_schaefer_2018
import networkx as nx
import bct

%matplotlib inline

We'll load in our atlas we used in tutorial 10 (the Schaefer 200 atlas). We'll be using this later. 

In [ ]:
atlas = fetch_atlas_schaefer_2018(n_rois=200, resolution_mm=2)
labels = [x.decode() for x in atlas['labels']]

## 1. Thresholding

First, we'll load our connectivity matrix and then threshold it using a proportional threshold of the top 10% of connectivity weights. All of the measures we'll use here work with either weighted or binary matrices (note that some methods that we don't explore only accept binary matrices). 

In [ ]:
cmat = np.loadtxt('connectivity.csv', delimiter=',')

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, vmax=1, vmin=-1, figure=fig);

In [ ]:
thresh_cmat = bct.threshold_proportional(cmat, 0.1)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_cmat, vmax=1, vmin=-1, figure=fig);

## 2. Constructing a graph

Let's just take the same subnetwork we looked at last week, and also add in some motor regions. This will make life a bit easier for understanding various graph theory concepts and measures. Selecting this set of regions from our thresholded matrix:

In [ ]:
A = thresh_cmat[:30, :30]

# some fancy python to clean up the names for visualization (don't worry about understanding this) 
region_labels = ["".join(x.split("_")[-2:]) for x in labels[:30]]

fig, ax = plt.subplots(figsize=(5, 5))
# plotting.plot_matrix(A, vmax=1, vmin=-1)
plotting.plot_matrix(A, vmax=1, vmin=-1, labels=region_labels, figure=fig);

Graphs represent the connectivity structure among the regions in this matrix. Each region is a **node**. And each corretion between regions/nodes are called **edges**; the strength of the correlation is the edge **weight**. First we need to build our graph in networkx (`nx`). We can use our connectivity matrix, called an **adjacency matrix** in graph theory (`A`), to create `G`, our graph. Note that we're generating an **undirected weighted** graph because correlations (weights) are nondirectional. 

We'll also label our nodes according to the regions. 

In [ ]:
G = nx.from_numpy_array(A)
G = nx.relabel_nodes(G, lambda x: region_labels[x])

### 2.1 Plotting our graph

Now we can plot our graph. A really common way to plot graphs are using force-directed drawing algorithms that try to position nodes such that the edges are roughly equal in length with minimal overlap (see [Wikipedia](https://en.wikipedia.org/wiki/Force-directed_graph_drawing)). Networkx has the Kamada-Kawai algorithm: 

In [ ]:
# create a list of colours to match the network labels, Vis (purple) and SomMot (blue)
region_colors = ['purple'] * 14 + ['steelblue'] * 16

nx.draw_kamada_kawai(G, node_color=region_colors, node_size=500, 
                     with_labels=True, font_color='w', font_size=5)

Go back and compare this visualization with the connectivity matrix. Does it make sense? Why are some regions close together while others are more spread out?

### 2.2 Adding weights

Above, we're completely ignoring weights/correlations. We can add these in, but we'll have to program this differently so we can customize it a bit.

First, we can extract out our edges and their corresponding weights. This is akin to going through each cell above and getting the corresponding regions, and the correlation value.

In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
edges

In [ ]:
weights

In [ ]:
weight_widths = [(.5 + x) ** 4 for x in weights]

layout = nx.kamada_kawai_layout(G)
nx.draw_networkx_nodes(G, layout, node_color=region_colors, node_size=500)
nx.draw_networkx_labels(G, layout, font_color='w', font_size=5)
nx.draw_networkx_edges(G, layout, edgelist=edges, width=weight_widths)
plt.axis('off');

## 3. Graph measures

Graph theory is the study of graphs and their properties. We can use graph theory measures to understand the connectivity structure in our data. For example, what region has the greatest number of connections? What region is the most 'central' to our network? For any two regions, how many other regions are 'in between'?

### 3.1 Network density

We can measure how dense each network is by dividing the number of edges by the number of possible edges. In other words, **network density** is the fraction of possible edges that are present.   

In [ ]:
vis_density, n, k = bct.density_und(A[:14, :14])
mot_density, n, k = bct.density_und(A[14:, 14:])

fig, ax = plt.subplots(figsize=(2, 3))
ax.bar(['Vis', 'Mot'], [vis_density, mot_density],
       width=.8, color=['purple', 'steelblue'])
sns.despine()

By looking at our connectivity matrix or graph, we can confirm these results. More motor regions are connected to one another than visual regions are.

### 3.2 Node degree and strength

**Node degree** is simply the number of edges for a node. If we didn't threshold our connectivity matrix, all nodes would have the same degree (i.e. number of regions - 1). Because we applied a threshold, certain nodes have greater degree than others. We can use BCT to compute the degrees of our nodes:  

In [ ]:
degrees = bct.degrees_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, degrees, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

Node degree ignores the connectivity weights (i.e. correlation). **Node strength**, however, is the sum of weights for a node. 

In [ ]:
strengths = bct.strengths_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, strengths, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.3 Path length

A **path** is a series of edges between nodes. The **shortest path** is, as the name implies, the shortest possible path between two nodes. Directly adjacent or connected nodes have a shortest path of 1, and indirectly connected nodes have a shortest path of > 1. Therefore, the shortest path is a measure of distance between pairs of nodes. 

We can obtain the shortest paths using Dijkstra's algorithm implemented in BCT. First, we'll plot the number of edges that make up the shortest path between each node. In an unweighted (binary) matrix, the number of edges is equivalent to the path length: 

In [ ]:
# need to convert A to inverse correlation or 'length' matrix
A_inv = bct.weight_conversion(A, 'lengths')

dist, num = bct.distance_wei(A_inv)

fig, ax = plt.subplots(figsize=(8, 8))
plotting.plot_matrix(num, vmin=0, labels=region_labels, 
                     figure=fig, cmap='viridis')
ax.set_title('Number of edges');

Because our matrix is weighted (i.e. has correlations), path length takes the edge strength into account. Stronger correlations are interpreted as shorter paths. In weighted matrix, the shortest path is the sum of inverse correlations:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plotting.plot_matrix(dist, vmin=0, labels=region_labels, 
                     figure=fig, cmap='viridis')
ax.set_title('Shortest weighted path');

We can also compute the average shortest path, otherwise known as the **characteristic path length**:

In [ ]:
# exclude infinite values (white squares above)
characteristic_path = np.mean(dist[~np.isinf(dist)])
characteristic_path

### 3.4 Centrality

Centrality speaks to the extent of which a node is 'central' to the network. There are a variety of measures for centrality, but a particularly popular measure is **betweeness centrality**. Betweeness centrality measures the proportion of shortest paths between all node pairs that pass through a node. If many paths pass through a node, that node is deemed central to a network. This will make a lot of sense when you compare these results with the graph above.

In [ ]:
# input the inverse matrix
between_cent = bct.betweenness_wei(A_inv)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, between_cent, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

Alternatively, there is the **eigenvector centrality**. Here, nodes are more central if they are connected to other central nodes. We expect more central nodes to be in the center of the network, and therefore connected to one another. As a thought experiment, how might this relate to network density?

In [ ]:
eigin_cent = bct.eigenvector_centrality_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, eigin_cent, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.5 Participation Coefficient

The **participant coefficient** is simply the number of edges that connect to nodes in different clusters/modules (e.g., brain network). So we can measure how much a brain region 'participates' in other brain networks. If the participation coefficient is high, the brain region connects to many other brain networks. Low participation coefficients mean that a brain region mostly (if not entirely) connects to regions within the network it belongs to. Any guesses as to what region has the highest participation coefficient?

In [ ]:
# label based on Vis or SomMot
network_index = [1] * 14 + [2] * 16
participation = bct.participation_coef(A, network_index)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, participation, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.6 Modularity

Modules refer to subgroups (or clusters) of densely connected nodes. We might expect our pre-defined functional brain networks that we already have (Vis, SomMot) are their own clusters. We can also apply clustering algorithms or dedicated community-detection algorithms to generate our own communities/clusters/modules. 

We've seen hierarchical clustering before when we did RSA and briefly when we were visualizing connectivity matrices last week. If we apply hierarchical clustering to these data, we get the following:

In [ ]:
# get upper triangle
distances = spatial.distance.squareform(1 - A, checks=False)

# apply hierarchical clustering 
linkages = cluster.hierarchy.linkage(distances, method='average')

# plot dendogram
dendo = cluster.hierarchy.dendrogram(linkages, labels=region_labels)
plt.xticks(rotation=45);

Hierarchical clustering is an agglomerative clustering techniques that forms clusters based on the closeness or similarity of nodes within a cluster.

with BCT, we can use the generalized Louvain algorithm to estimate communities within the data. The Louvain algorithm is an algorithm that iteratively assigns nodes to different clusters/communities and converges on the solution that gives the highest modularity metric, _Q_. _Q_ indicates the extent to which the network is cleanly delineated into groups. When we run this function, we'll get our module label for each node, and _Q_.

In [ ]:
module_louvain, q = bct.community_louvain(A)
q

Now we visualize the generalized Louvain algorithm results.

In [ ]:
nx.draw_kamada_kawai(G, node_color=module_louvain, node_size=500, 
                     with_labels=True, font_color='w', font_size=5)

## 4. Whole-brain analysis

Now that we've explore some metrics in our subnetwork, we can apply some of them to the whole brain. For simplicity, let's assign each region a number so that we have a reference when visualizing our data.

In [ ]:
label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region': labels})
label_data

In [ ]:
label_data.query("number == 30")

### 4.1 Constructing a graph

We'll construct a whole-brain graph. Before we do that, just a couple of functions that assigns our nodes a color and network name. 

In [62]:
def set_color(x):
    
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 
                'Limbic', 'Cont', 'Default']
    cmap = ['purple', 'steelblue', 'green', 'violet', 
            'lightgoldenrodyellow', 'orange', 'indianred']
    pairings = dict(zip(networks, cmap))
    
    network_label = x.split('_')[2]
    return pairings[network_label]

def set_network(x):
    
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 
                'Limbic', 'Cont', 'Default']
    index = [1, 2, 3, 4, 5, 6, 7]
    pairings = dict(zip(networks, index))
    
    network_label = x.split('_')[2]
    return pairings[network_label]

node_colors = [set_color(i) for i in labels]
node_network = [set_network(i) for i in labels]

In [ ]:
G = nx.from_numpy_array(thresh_cmat)
G = nx.relabel_nodes(G, lambda x: label_data['number'].tolist()[x])

In [ ]:
nx.draw_kamada_kawai(G, node_color=node_colors, node_size=150, with_labels=True, font_size=5)

Talk about a mess! This kind of looks like a hairball, but you can see that there is structure there. Regions belonging the the same networks, particularly the purple (Vis), blue (SomMot), and Red (Default) tend to stick together.

For reference, the Schaefer atlas layout is below: 

![](https://github.com/danjgale/psyc-917/blob/master/images/schaefer_100.png?raw=true)

### 4.2 Graph metrics 

Now we can run some of our graph measures we ran above.

In [81]:
# change A to entire brain
A = thresh_cmat

Lets visualize the connectivity distance between regions.

In [ ]:
A_inv = bct.weight_conversion(A, 'lengths')

dist, num = bct.distance_wei(A_inv)

fig, ax = plt.subplots(figsize=(8, 8))
plotting.plot_matrix(dist, vmin=0,
                     figure=fig, cmap='viridis')
ax.set_title('Connectivity distance');

Now, lets visualize some stats on the brain. As we've seen, we can compute measures that are properties of a brain region, like the number of connections it has. What does this look like on the brain itself? We'll create a function to plot this data on the brain as a stat map, like we've seen before. This basically maps the region's value to every voxel that belongs to the region, therefore visualizing each region's value. 

In [83]:
def stat_map(x, atlas):

    img_data = atlas.get_fdata()
    indices = np.unique(img_data)[1:]

    arr = img_data.copy()
    for val, i in zip(x, indices):
        arr = np.where(arr == i, val, arr)
        
    return nib.Nifti1Image(arr, atlas.affine)

Now lets compute a bunch of measures:

In [84]:
degrees = bct.degrees_und(A)
strengths = bct.strengths_und(A)
eigin_cent = bct.eigenvector_centrality_und(A)
between_cent = bct.betweenness_wei(A_inv)

participation = bct.participation_coef(A, node_network)

We can visualize any of these measures with the code below:

In [ ]:
measure = between_cent # change out with any of the above!

atlas_img = nib.load(atlas['maps'])

res = stat_map(measure, atlas_img)
plotting.view_img(res, vmin=0, cmap='magma', symmetric_cmap=False)